In [1]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
# Load the dataset
df = pd.read_csv('whales/train.csv')
image_ids = df['Image'].tolist()
whale_ids = df['whaleID'].tolist()

# Define image size and batch size
IMG_SIZE = (128,128)


# Define a function to preprocess images


In [3]:
def preprocess_image(image_path):
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=IMG_SIZE)
    image = tf.keras.preprocessing.image.img_to_array(image)
    return image

# Preprocess images
X = np.array([preprocess_image(f"whales/imgs_no_bg/{img_id}") for img_id in image_ids])
Y = np.array(whale_ids)

In [4]:
# One-hot encode the labels
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
Y_encoded = label_encoder.fit_transform(Y)

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y_encoded, test_size=0.2, random_state=42)

# One-hot encode the labels for Keras
Y_train_onehot = tf.keras.utils.to_categorical(Y_train, num_classes=len(np.unique(Y_encoded)))
Y_test_onehot = tf.keras.utils.to_categorical(Y_test, num_classes=len(np.unique(Y_encoded)))
# Y_train_onehot = Y_train
# Y_test_onehot = Y_test


In [6]:
# Load the pre-trained MobileNetV2 model, without the top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128,128, 3))

# Freeze the base model layers (optional)
base_model.trainable = False

# Add custom layers on top of the base model
x = base_model.output
x = GlobalAveragePooling2D()(x)  # Add a global spatial average pooling layer
x = Dense(1024, activation='relu')(x)
x=Dense(512, activation='relu')(x)
x = Dropout(0.2)(x)
x = Dense(256, activation='relu')(x)
x=Dense(128, activation='relu')(x)
x = Dropout(0.2)(x)
predictions = Dense(len(np.unique(Y_encoded)), activation='softmax')(x)  # Final classification layer

# Create the full model


In [7]:
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
BATCH_SIZE = 1
history = model.fit(X_train, Y_train_onehot, validation_data=(X_test, Y_test_onehot), epochs=100, batch_size=BATCH_SIZE)

# Optionally, unfreeze some of the base layers and fine-tune the model
# Unfreeze the base model layers
base_model.trainable = True


Epoch 1/100
2907/2907 [==============================] - 47s 12ms/step - loss: 6.0649 - accuracy: 0.0072 - val_loss: 6.0330 - val_accuracy: 0.0041
Epoch 2/100
2907/2907 [==============================] - 32s 11ms/step - loss: 5.9371 - accuracy: 0.0076 - val_loss: 6.0426 - val_accuracy: 0.0069
Epoch 3/100
2907/2907 [==============================] - 32s 11ms/step - loss: 5.9019 - accuracy: 0.0089 - val_loss: 6.1220 - val_accuracy: 0.0055
Epoch 4/100
2907/2907 [==============================] - 33s 11ms/step - loss: 5.8931 - accuracy: 0.0107 - val_loss: 6.1022 - val_accuracy: 0.0069
Epoch 5/100
2907/2907 [==============================] - 32s 11ms/step - loss: 5.8770 - accuracy: 0.0103 - val_loss: 6.1045 - val_accuracy: 0.0069
Epoch 6/100
2907/2907 [==============================] - 33s 11ms/step - loss: 5.8745 - accuracy: 0.0114 - val_loss: 6.1514 - val_accuracy: 0.0069
Epoch 7/100
2907/2907 [==============================] - 33s 12ms/step - loss: 5.8606 - accuracy: 0.0107 - val_loss: 6

KeyboardInterrupt: 

In [ ]:
# Recompile and fine-tune the model
model.compile(optimizer=tf.keras.optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
history_fine_tune = model.fit(X_train, Y_train_onehot, validation_data=(X_test, Y_test_onehot), epochs=5, batch_size=BATCH_SIZE)
